In [14]:
# import module
import random
import datetime

for i in range(0, 11):
    ct = datetime.datetime.now()
    random.seed(a=ct.timestamp(),version=2)
    print(int(random.random()*10000))

1024
5436
9470
6730
5974
6523
6891
3552
7849
5490
5225
